In [3]:
from __future__ import division
import argparse
import tensorflow as tf
from tensorflow.keras import layers

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras

print(tf.VERSION)
print(tf.keras.__version__)

import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact
from PIL import Image
import os
from glob import glob
import seaborn as sns
import cv2
from keras.utils.np_utils import to_categorical 
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, LSTM, TimeDistributed
from keras import backend as K
import itertools
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.utils.np_utils import to_categorical 
from sklearn.model_selection import train_test_split
import re

1.14.0
2.2.4-tf


Using TensorFlow backend.


# Functions for Reading in Images

In [4]:
# function for reading in images + creating labels given image directory and classes:

def read_images1(image_dir, classes1):
    imageid_path = image_dir
    c1 = classes1
    
    
    class_dict = {}
    for x in imageid_path:
        for typ in c1:
            pattern = re.compile(typ)
            if pattern.search(x):
                class_dict[imageid_path[x]] = typ 
    
    return  class_dict

In [5]:
# function for reading in images + creating labels given image directory and classes:

def read_images2(image_dir, classes1):
    imageid_path = image_dir
    c1 = classes1
    
    class_dict = {}
    for x in imageid_path:
        f_1 = os.path.split(imageid_path[x])
        f_2 = os.path.split(f_1[0])
        class_dict[imageid_path[x]] = f_2[1] 
    
    for x in imageid_path:
        
        if class_dict[imageid_path[x]] == "sunny":
            class_dict[imageid_path[x]] = "shine"
    
    
    return class_dict

In [ ]:
def labels_quick(image_dir, class_dict, classes_final):
    c2 = classes_final
    op = []
    for x in image_dir:
        op.append(class_dict[image_dir[x]])
        
    labels = np.asarray(op)
    labelslist = (labels.tolist())
    
    labels_num = []
    for x in labelslist:
        for i in range(len(c2)):
            if x == c2[i]:
                labels_num.append(np.asarray([i]))
                
            
    
    labels_num = np.asarray(labels_num)
    print(labels_num.shape)
    return labels_num, labelslist

In [ ]:
def labels_quick_order(image_dir, class_dict, classes_final):
    c2 = classes_final
    op = []
    for x in sorted(image_dir):
        op.append(class_dict[image_dir[x]])
        
    labels = np.asarray(op)
    labelslist = (labels.tolist())
    
    labels_num = []
    for x in labelslist:
        for i in range(len(c2)):
            if x == c2[i]:
                labels_num.append(np.asarray([i]))
                
            
    
    labels_num = np.asarray(labels_num)
    print(labels_num.shape)
    return labels_num, labelslist

In [6]:
def create_image_data(image_dir, classes_final, class_dict):
    imageid_path = image_dir
    c2 = classes_final
    data_a = []
    for x in imageid_path:
        if class_dict[imageid_path[x]] in c2:
            oimg = cv2.imread(imageid_path[x])[...,::-1]
            nimg = cv2.resize(oimg,(100,75))
            data_a.append(nimg)
    data = np.asarray(data_a)
    print(data.shape)
    return data

In [ ]:
def create_image_data_order(image_dir, classes_final, class_dict):
    imageid_path = image_dir
    c2 = classes_final
    data_a = []
    for x in sorted(imageid_path):
        if class_dict[imageid_path[x]] in c2:
            oimg = cv2.imread(imageid_path[x])[...,::-1]
            nimg = cv2.resize(oimg,(100,75))
            data_a.append(nimg)
    data = np.asarray(data_a)
    print(data.shape)
    return data

In [7]:
def breakdown(lablist, classes2): #lablist is the labels as a list, classes2 is the final number 
    
    total = 0
    for typ in classes2:
       
        total += lablist.count(typ)


    for typ in classes2:
        print( typ ,":", lablist.count(typ), round(lablist.count(typ)/total, 3))
    print(total)   

In [8]:
# function for creating subset of an image dataset, will pass a list of the total number of images desired for each category


def create_subset(image_dir, class_dict, classes_final, limits):
    
    subset = {}
    counts = np.zeros(len(limits)).tolist()

    for x in image_dir:
        for i in range(len(limits)):
            
            if class_dict[image_dir[x]] == classes_final[i]:
                counts[i] = counts[i]+1
                if counts[i] <(limits[i]+1):
                    subset[x] = image_dir[x]
    
    
    
    class_subset = {} #dictionary of classes associated with each image is created
    for x in subset:
        class_subset[subset[x]] = class_dict[subset[x]] 
        
    op = []
    for x in subset:
        op.append(class_subset[subset[x]])
    
    labels_subset = np.asarray(op)
    labels_list_subset = (labels_subset.tolist())
    
    labels_num = []
    for x in labels_list_subset:
        for i in range(len(classes_final)):
            if x == classes_final[i]:
                labels_num.append(np.asarray([i]))
    labels_num = np.asarray(labels_num)
    
    
    print( labels_num.shape)
    return  subset, labels_num , labels_list_subset , class_subset

In [ ]:
# function for creating subset of an image dataset, will pass a list of the total number of images desired for each category


def create_subset_order(image_dir, class_dict, classes_final, limits):
    
    subset = {}
    counts = np.zeros(len(limits)).tolist()

    for x in sorted(image_dir):
        for i in range(len(limits)):
            
            if class_dict[image_dir[x]] == classes_final[i]:
                counts[i] = counts[i]+1
                if counts[i] <(limits[i]+1):
                    subset[x] = image_dir[x]
    
    
    
    class_subset = {} #dictionary of classes associated with each image is created
    for x in subset:
        class_subset[subset[x]] = class_dict[subset[x]] 
        
    op = []
    for x in sorted(subset):
        op.append(class_subset[subset[x]])
    
    labels_subset = np.asarray(op)
    labels_list_subset = (labels_subset.tolist())
    
    labels_num = []
    for x in labels_list_subset:
        for i in range(len(classes_final)):
            if x == classes_final[i]:
                labels_num.append(np.asarray([i]))
    labels_num = np.asarray(labels_num)
    
    
    print( labels_num.shape)
    return  subset, labels_num , labels_list_subset , class_subset

In [ ]:
def subplots(data, label_list):
    plt.figure(figsize=(10,10))
    for i in range(30):
        plt.subplot(5,6,i+1)
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        plt.imshow(data[i], cmap=plt.cm.binary)
        plt.xlabel(label_list[i])

In [ ]:
def combine(image_dir1, image_dir2, class_dict1,class_dict2):
    images = {}
    labels = {}
    for x in image_dir1:
        images[x] = image_dir1[x]
        labels[image_dir1[x]] = class_dict1[image_dir1[x]]
    
    for x in image_dir2:
        images[x] = image_dir2[x]
        labels[image_dir2[x]] = class_dict2[image_dir2[x]]
        
    return images, labels

# Functions for Model Training

In [ ]:
def normalize(data_o):
    data_o_mean = np.mean(data_o)
    data_o_std = np.std(data_o)
    data = (data_o - data_o_mean)/data_o_std
    
    return data

In [ ]:
def plots1(history):
    history_dict = history.history
    history_dict.keys()
    
    acc = history_dict['acc']
    val_acc = history_dict['val_acc']
    loss = history_dict['loss']
    val_loss = history_dict['val_loss']

    epochs = history.epoch

    plt.plot(epochs, loss, 'bo', label='Training loss')
    plt.plot(epochs, val_loss, 'b', label='Validation loss')
    plt.title('Training and validation loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

In [ ]:
def plots2(history):
    history_dict = history.history
    history_dict.keys()
    
    acc = history_dict['acc']
    val_acc = history_dict['val_acc']
    loss = history_dict['loss']
    val_loss = history_dict['val_loss']

    epochs = history.epoch
    
    plt.plot(epochs, acc, 'bo', label='Training acc')
    plt.plot(epochs, val_acc, 'b', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()